In [4]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import base64
from PyPDF2 import PdfFileReader
from io import BytesIO
from PyPDF2 import PdfReader
import re
from email.mime.text import MIMEText
import json

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import base64
import email

try:
    with open('processed_ids.json', 'r') as f:
        processed_ids = set(json.load(f))
except FileNotFoundError:
    processed_ids = set()


processed_ids = set()  # Initialize an empty set at the beginning of your script

def get_sent_ids(service, to_email='Receipts@DivvyPay.com'):
    sent_ids = set()
    results = service.users().messages().list(userId='me', q=f"to:{to_email}").execute()
    messages = results.get('messages', [])
    for message in messages:
        sent_ids.add(message['id'])
    return sent_ids


def forward_email(service, user_id, msg_id, to_email):
    # Get the email message
    msg = service.users().messages().get(userId=user_id, id=msg_id, format='raw').execute()

    # Decode the raw email message
    msg_str = base64.urlsafe_b64decode(msg['raw']).decode()
    
    # Create a MIME message object from the raw email
    mime_msg = email.message_from_string(msg_str)

    # Create a new MIME message to forward
    msg_forward = MIMEMultipart()

    msg_forward['to'] = to_email
    msg_forward['from'] = user_id
    msg_forward['subject'] = f"Fwd: {mime_msg['subject']}"

    # Attach the text parts
    for part in mime_msg.walk():
        content_type = part.get_content_type()
        content_disposition = str(part.get("Content-Disposition"))

        try:
            if "attachment" not in content_disposition:
                # Download and process text/plain attachments
                if content_type == "text/plain":
                    msg_forward.attach(MIMEText(part.get_payload(decode=True).decode('utf-8'), 'plain'))
                elif content_type == "text/html":
                    msg_forward.attach(MIMEText(part.get_payload(decode=True).decode('utf-8'), 'html'))
            else:
                msg_forward.attach(part)
        except UnicodeDecodeError:
            print(f"Could not decode one of the parts in email id: {msg_id}. Skipping that part.")
            continue

        # Skip any text/plain (txt) attachments
        if "attachment" not in content_disposition:
            # Download and process text/plain attachments
            if content_type == "text/plain":
                msg_forward.attach(MIMEText(part.get_payload(decode=True).decode('utf-8'), 'plain'))
            elif content_type == "text/html":
                msg_forward.attach(MIMEText(part.get_payload(decode=True).decode('utf-8'), 'html'))

    # Attach the original attachments
    for part in mime_msg.walk():
        if part.get_content_maintype() == 'multipart':
            continue
        if part.get('Content-Disposition') is None:
            continue

        filename = part.get_filename()
        if bool(filename):
            attachment = MIMEApplication(part.get_payload(decode=True), Name=filename)
            msg_forward.attach(attachment)

    # Send the forwarded message
    raw_message = base64.urlsafe_b64encode(msg_forward.as_bytes()).decode("utf-8")
    raw_message = {'raw': raw_message}
    service.users().messages().send(userId=user_id, body=raw_message).execute()


# Initialize Gmail API
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
creds = None
if creds and not creds.valid:
    if creds.expired and creds.refresh_token:
        creds.refresh(Request())
else:
    flow = InstalledAppFlow.from_client_secrets_file('C:\\Users\\ryanv\\OneDrive\\Desktop\\credentials.json', ['https://www.googleapis.com/auth/gmail.readonly', 'https://www.googleapis.com/auth/gmail.send'])
    creds = flow.run_local_server(port=0)
    
service = build('gmail', 'v1', credentials=creds)

sent_ids = get_sent_ids(service, 'Receipts@DivvyPay.com')

# ... (previous code for Gmail API authentication and message retrieval)

from PyPDF2 import PdfReader

# ... (previous code for Gmail API authentication and initialization)
from PyPDF2 import PdfReader

# ... (previous code for Gmail API authentication and initialization)
from PyPDF2 import PdfReader

# Initialize service
service = build('gmail', 'v1', credentials=creds)
profile = service.users().getProfile(userId='me').execute()
my_email = profile.get('emailAddress')

# Initialize variables
page_token = None
total_found = 0  # Total number of invoice/receipt emails found
total_skipped = 0  # Total number of invoice/receipt emails skipped (already forwarded)
total_forwarded = 0  # Total number of invoice/receipt emails actually forwarded this time

# Get sent message IDs
sent_ids = get_sent_ids(service, 'Receipts@DivvyPay.com')

# Main loop for reading and forwarding emails
while True:
    results = service.users().messages().list(userId='me', pageToken=page_token).execute()
    
    messages = results.get('messages', [])
    if not messages:
        print('No messages found.')
        break

    for message in messages:
        msg_id = message['id']
        
        # Skip already processed messages
        if msg_id in processed_ids:
            print(f"Already processed email with ID: {msg_id}. Skipping.")
            total_skipped += 1
            continue
            
        msg = service.users().messages().get(userId='me', id=message['id']).execute()
        sender = ""
        subject = ""
        date = ""
        headers = msg['payload'].get('headers', [])
        
        for header in headers:
            name = header.get('name', '')
            if name == 'From':
                sender = header.get('value', '')
            elif name == 'Subject':
                subject = header.get('value', '')
            elif name == 'Date':
                date = header.get('value', '')
        
        # Skip the email if it's from receipts@divvypay.com or from the user's own email
        if 'receipts@divvypay.com' in sender.lower() or my_email.lower() in sender.lower():
            continue
        
        found_invoice_or_receipt = False
        # Use Regular Expressions or other string matching to check subject line or body
        if re.search(r'(invoice|receipt|payment confirmation|order confirmation|paid|order confirmed|amount due|order)', subject, re.I):
            found_invoice_or_receipt = True
        elif 'parts' in msg['payload']:
            for part in msg['payload']['parts']:
                if part['filename'].endswith('.pdf'):
                    attachment = service.users().messages().attachments().get(
                        userId='me', messageId=message['id'], id=part['body']['attachmentId']).execute()
                    file_data = base64.urlsafe_b64decode(attachment['data'].encode('UTF-8'))

                    with BytesIO(file_data) as f_io:
                        reader = PdfReader(f_io)
                        for page_num in range(len(reader.pages)):
                            text = reader.pages[page_num].extract_text()
                            if re.search(r'(invoice|receipt|payment confirmation|order confirmation|paid|order confirmed|amount due|order)', text, re.I):
                                found_invoice_or_receipt = True
                                break

        if found_invoice_or_receipt:
            total_found += 1
            print(f"Date: {date}, Sender: {sender}, Subject: {subject}")
            forward_email(service, 'me', message['id'], 'Receipts@DivvyPay.com')
            sent_ids.add(msg_id)  # You might choose to add this ID to sent_ids for the duration of the script
            total_forwarded += 1
            
        
        processed_ids.add(msg_id)  # Add the ID to processed_ids after forwarding or skipping

    # For pagination
    page_token = results.get('nextPageToken')
    if not page_token:
        break
        
# Print summary statistics
print(f"Total number of receipts and invoices found: {total_found}")
print(f"Total number of receipts and invoices already forwarded and filtered out: {total_skipped}")
print(f"Total number of new receipts and invoices forwarded: {total_forwarded}")

# Save processed_ids to a JSON file
with open('processed_ids.json', 'w') as f:
            json.dump(list(processed_ids), f)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=159341088951-ugoco8a9ajdpfovtcn03do44ht8rhoh4.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A57095%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=JSRruxU5FyhKX7uWAJ7arZu0eB33Sn&access_type=offline
Date: Sat, 02 Sep 2023 00:49:34 +0000, Sender: Sales <info@justmedicalstore.com>, Subject: Your Just Medical Store order confirmation
Date: Sat, 02 Sep 2023 00:49:36 +0000, Sender: Sales <info@justmedicalstore.com>, Subject: Invoice for your Just Medical Store order
Date: Thu, 31 Aug 2023 17:05:14 +0000, Sender: "Pipette.com Customer Support" <info@pipette.com>, Subject: Re: Your Pipette.com Order: #17723
Date: Mon, 28 Aug 2023 15:37:33 -0400, Sender: Nanocs <sales@nanocs.com>, Subject: Re: Card doesn't work(#29127)
Date: Mon, 28 Aug 2023 07:57:42 -0700, Sender: Kathleen Malek

KeyboardInterrupt: 